 ## Solving Capacitated Lot Sizing Problem using CPLEX's python module DOCPLEX

In [33]:
import random as r
import numpy as np

r.seed(50)

no_of_items = 10
no_of_machines = 2
no_of_period = 4

# setting up the cost
low_setup_time = np.asarray([int(r.uniform(a=10,b=50)) for i in range(no_of_items*no_of_machines)])
high_setup_time = 1.5*low_setup_time

low_setup_cost = np.asarray([int(r.uniform(a=5,b=95)) for i in range(no_of_items*no_of_machines)])
high_setup_cost = 10*low_setup_cost

unit_production_cost = [round(r.uniform(a=1.5,b=2.5), 2) for i in range(no_of_items*no_of_machines)]
unit_inventory_cost = [round(r.uniform(a=0.2,b=0.4),2) for i in range(no_of_items*no_of_period)]
unit_production_time = [int(r.uniform(a=1,b=5)) for i in range(no_of_items*no_of_machines)]

demand = [round(r.uniform(a=1,b=180)) for i in range(no_of_items*no_of_period)]

In [34]:
# Assigning Costs:

Item_list = [i for i in range(1,no_of_items+1)]
Machine_list = [j for j in range(1,no_of_machines+1)]
Period_list = [t for t in range(1,no_of_period+1)]

item_machine_pair = [(i,j) for i in Item_list for j in Machine_list]
item_period_pair = [(i,t) for i in Item_list for t in Period_list]
machine_period_pair = [(j,t) for j in Machine_list for t in Period_list]

Bij = {(i,j): s for (i,j),s in zip(item_machine_pair,unit_production_time)}   # Unit production time of item i on machine j;
Cij = {(i,j): s for (i,j),s in zip(item_machine_pair,unit_production_cost)}   # Unit production cost of item i on machine j;
Hit = {(i,t): s for (i,t),s in zip(item_period_pair,unit_inventory_cost)}     # Unit inventory cost of item i per period t;
Dit = {(i,t): s for (i,t),s in zip(item_period_pair,demand)}                  # Demand of item i at period t;
Fij = {(i,j): s for (i,j),s in zip(item_machine_pair,low_setup_time)}         # Setup time of item i on machine j;
Sij = {(i,j): s for (i,j),s in zip(item_machine_pair,low_setup_cost)}         # Setup cost of item i on machine j.

In [35]:
# Instanciating the variables:

Vars = []
for items in Item_list:
    for machines in Machine_list:
        for periods in Period_list:
            Vars.append((items,machines,periods))

In [36]:
from docplex.mp.model import Model

In [37]:
# Initializing the Model:

mip = Model('Capacitated Lot Sizing Problem')

In [38]:
# Declaring decision variables:

X = mip.integer_var_dict(Vars, lb=0, name='x')
Y = mip.binary_var_dict(Vars,name='y')
S = mip.integer_var_dict(item_period_pair, lb=0, name='s')

In [39]:
# Averaging out demand of each item per period in order to calculate capacity of each machine per period:

Cap = mip.sum(((Dit[i,t]/no_of_machines)*Bij[i,j]) + Fij[i,j] for i,j,t in X)                       
capacity_per_period = []
for i in range(1,no_of_period+1):
    Q = int((Cap/(no_of_machines*i)).constant)      
    capacity_per_period.append(Q)

Qjt = {(j,t): s for (j,t),s in zip(machine_period_pair,capacity_per_period*no_of_period)}           # Capacity of machine j at period t;
M = max(capacity_per_period)                                                                        # Maximum capacity of machines per period.

In [40]:
cost_list = [Bij,Cij,Hit,Dit,Fij,Sij,Qjt,M]

In [41]:
# Constraints:

# constraint_1 ensure that the demand of each item in each period should be satisfied 
constraint_1 = mip.add_constraints(mip.sum(X[i,j,t] for j in range(1,no_of_machines+1)) + (-S[i,t] + S[i,t-1] if t-1 > 0 else -S[i,t]) == Dit[i,t] for i,t in S)

# constraint_2 restrict the production capacity for each machine j at each period t
constraint_2 = mip.add_constraints(mip.sum(Bij[i,j]*X[i,j,t] + Fij[i,j]*Y[i,j,t] for i in range(1,no_of_items+1)) <= Qjt[j,t] for j,t in Qjt)

# constraint_3 impose the setup cost on machine j at period t for item i when Xijt is positive.
constraint_3 = mip.add_constraints(X[i,j,t] <= M*Y[i,j,t] for i,j,t in X)

In [42]:
# Objective function:

setup_cost = mip.sum(Sij[i,j]*Y[i,j,t] for i,j,t in Y) 
production_cost = mip.sum(Cij[i,j]*X[i,j,t] for i,j,t in X)
inventory_cost = mip.sum(Hit[i,t]*S[i,t] for i,t in S)
total_cost = setup_cost + production_cost + inventory_cost

In [43]:
# Goal:

mip.minimize(total_cost)

In [44]:
mip.print_information()

Model: Capacitated Lot Sizing Problem
 - number of variables: 200
   - binary=80, integer=120, continuous=0
 - number of constraints: 128
   - linear=128
 - parameters: defaults
 - objective: minimize
 - problem type is: MILP


In [45]:
solution = mip.solve(log_output=True)
assert solution is not None
value = solution.objective_value

CPXPARAM_Read_DataCheck                          1
Found incumbent of value 8563.040000 after 0.00 sec. (0.02 ticks)
Tried aggregator 1 time.
MIP Presolve eliminated 10 rows and 0 columns.
MIP Presolve added 20 rows and 0 columns.
MIP Presolve modified 174 coefficients.
Reduced MIP has 138 rows, 200 columns, and 490 nonzeros.
Reduced MIP has 80 binaries, 120 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.53 ticks)
Probing fixed 0 vars, tightened 76 bounds.
Probing time = 0.00 sec. (0.24 ticks)
Tried aggregator 2 times.
MIP Presolve modified 30 coefficients.
Aggregator did 1 substitutions.
Reduced MIP has 137 rows, 199 columns, and 487 nonzeros.
Reduced MIP has 80 binaries, 119 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.48 ticks)
Probing fixed 0 vars, tightened 5 bounds.
Probing time = 0.00 sec. (0.19 ticks)
Clique table members: 10.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, 

In [46]:
print("The objective function value is: {}".format(round(value,2)))

The objective function value is: 8106.73


In [47]:
print("Obtained solution details:\n{}and it was solved by: {} solver".format(solution.solve_details,solution.solved_by))

Obtained solution details:
status  = integer optimal, tolerance
time    = 0.125 s.
problem = MILP
gap     = 0.0065785%
and it was solved by: cplex_local solver


In [48]:
solution_y = solution.get_value_dict(Y)
nonzero_allocation = []
for keys in solution_y:
    if solution_y[keys] == 1:
        nonzero_allocation.append(keys)
    else:
        pass
solution_x = solution.get_value_dict(X)
nonzero_amount = []
for keys in solution_x:
    if solution_x[keys] != 0:
        nonzero_amount.append(keys)
    else:
        pass
if nonzero_allocation == nonzero_amount: print("All cases matched.")

All cases matched.


#### *Solving with Lagrangian Relaxation*

In [50]:
from ipynb.fs.full.Lagrangian_relaxed_model import Lagrangian_relaxation
output = Lagrangian_relaxation(cost_list,Vars,no_of_items,no_of_machines,no_of_period)

print("\nLower bound of the problem = {}".format(output.objective_value))

1> new lagrangian iteration:
	 obj=17529, m=[1, 1, 1, 1, 1, 1, 1, 1], p=[0, 0, 0, 0, 0, 0, 0, 3.0]
1> -- loop continues, m=[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0], justifier=3
2> new lagrangian iteration:
	 obj=17529, m=[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0], p=[0, 0, 0, 0, 0, 0, 0, 3.0]
* Lagrangian relaxation succeeds, best_solution=17529, penalty=3, #iterations=2

Lower bound of the problem = 17529.01


In [51]:
output.export_as_mst(basename="%s",write_level= 4, path="D:/Study Docs/Sem 2/Seminar/CLSP")

'D:/Study Docs/Sem 2/Seminar/CLSP\\clsp_-with-_lagrangian_relaxation.mst'

In [52]:
solution.clear()

#### *Solving the classical with lower bound solution obtained from Lagrangian relaxed model*

In [53]:
relaxed_solution = mip.read_mip_starts("clsp_-with-_lagrangian_relaxation.mst")[0][0]
mip.add_mip_start(relaxed_solution)

In [55]:
new_solution = mip.solve(log_output=True)
new_obj_val = new_solution.objective_value
if new_obj_val == value: print("Optimal!")
print(new_obj_val)

CPXPARAM_Read_DataCheck                          1
2 of 9 MIP starts provided solutions.
MIP start 'm8' defined initial solution with objective 17526.0100.

Root node processing (before b&c):
  Real time             =    0.00 sec. (0.11 ticks)
Parallel b&c, 8 threads:
  Real time             =    0.00 sec. (0.00 ticks)
  Sync time (average)   =    0.00 sec.
  Wait time (average)   =    0.00 sec.
                          ------------
Total (root+branch&cut) =    0.00 sec. (0.11 ticks)
Optimal!
8106.730000000002


#### Summary: Solution Obtained is optimal since Lower Bound equals the obtained solution.